In [147]:
import pandas as pd

def read_data(file_path):
    df = pd.read_csv(file_path, header=[0,1], index_col=0)
    return df

df = read_data("./NOWHERE_DATASET_copy.csv")
df.head(5)

Name  Year GEOGRAPHY                                            \
        Name  Year    Europe Nrth America Middle East Asia Sth America   
0  CH-1995-1  1995         1            0           0    0           0   
1  CH-1995-3  1995         1            0           0    0           0   
2   E-1995-4  1995         1            0           0    0           0   
3   E-1995-7  1995         1            0           0    0           0   
4  E-1995-19  1995         1            0           0    0           0   

  REALITY                  ... CONTENT TO ME                        \
     Void Non-place Space  ...    Convention Struggle Interference   
0       1       1.0   0.0  ...           0.0      0.0          0.0   
1       0       0.0   0.0  ...           0.5      0.0          0.0   
2       0       0.5   0.0  ...           0.0      0.0          0.0   
3       0       0.0   0.0  ...           0.0      0.0          0.0   
4       0       0.0   0.0  ...           0.0      0.0          0.0   

                                                                              \
  Substitution Alienation Space & Time Pretention Addiction Belief/disbelief   
0          0.0        0.0          1.0        0.0       0.0                0   
1          0.0        0.0          0.0        0.3       0.0                0   
2          0.0        0.0          0.0        0.0       0.0                0   
3          0.0        0.0          0.0        0.0       0.0                0   
4          0.0        0.0          0.0        0.0       0.0                0   

             
  High/Kick  
0       0.0  
1       0.0  
2       0.0  
3       0.0  
4       0.0  

[5 rows x 149 columns]

In [148]:
def get_feature_names(data):
    """
    Gets the feature names for graph labelling purposses
    returns a list of all feature names
    """

    names = []
    for name in data.columns:
        s = ''
        s += name[0] 
        if name[0] != name[1]:
            s += ":" + " " + name[1]
        names.append(s)
        
    return names

feature_names = get_feature_names(df)

In [198]:
def TSNE_preprocess(data):
    """
    Preprocess for TSNE so that only columns with numeric values remain
    return a dataframe with only numeric columns
    """
        
    for d_type, d_column in zip(data.dtypes, data.columns):
        
        if d_type == "object":
            data = data.drop(columns=[d_column])
            
        if d_column == ('Year', 'Year'):
            data = data.drop(columns=[d_column])
            
    return data

df_processed = TSNE_preprocess(df)
df_processed.head(5)

GEOGRAPHY                                             REALITY               \
     Europe Nrth America Middle East Asia Sth America Non-place Space Nature   
0         1            0           0    0           0       1.0   0.0    1.0   
1         1            0           0    0           0       0.0   0.0    0.5   
2         1            0           0    0           0       0.5   0.0    0.5   
3         1            0           0    0           0       0.0   0.0    0.0   
4         1            0           0    0           0       0.0   0.0    0.0   

                        ... CONTENT TO ME                                   \
  Development Suburbia  ...    Limitation Convention Struggle Interference   
0         0.0      0.0  ...           0.0        0.0      0.0          0.0   
1         0.5      0.5  ...           0.0        0.5      0.0          0.0   
2         1.0      0.0  ...           0.0        0.0      0.0          0.0   
3         0.0      0.0  ...           0.0        0.0      0.0          0.0   
4         0.0      0.0  ...           0.0        0.0      0.0          0.0   

                                                                       
  Substitution Alienation Space & Time Pretention Addiction High/Kick  
0          0.0        0.0          1.0        0.0       0.0       0.0  
1          0.0        0.0          0.0        0.3       0.0       0.0  
2          0.0        0.0          0.0        0.0       0.0       0.0  
3          0.0        0.0          0.0        0.0       0.0       0.0  
4          0.0        0.0          0.0        0.0       0.0       0.0  

[5 rows x 145 columns]

In [199]:
from sklearn.manifold import TSNE

def do_TSNE(data, output_dims, verbose=2, perplexity=30):
    """
    Reduce the dimensionaly from data.colmns into to output_dims
    Returns a numpy array of (len(data), output_dims)    
    """
    
    tsne = TSNE(n_components=output_dims, random_state=0, verbose=verbose, perplexity=perplexity)
    data_2d = tsne.fit_transform(data)
    
    return data_2d
    

results = do_TSNE(df_processed, 2)    

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 220 samples in 0.001s...
[t-SNE] Computed neighbors for 220 samples in 0.011s...
[t-SNE] Computed conditional probabilities for sample 220 / 220
[t-SNE] Mean sigma: 1.081037
[t-SNE] Computed conditional probabilities in 0.009s
[t-SNE] Iteration 50: error = 64.0613327, gradient norm = 0.4734308 (50 iterations in 0.067s)
[t-SNE] Iteration 100: error = 63.5082169, gradient norm = 0.4877102 (50 iterations in 0.061s)
[t-SNE] Iteration 150: error = 62.4215050, gradient norm = 0.4836229 (50 iterations in 0.058s)
[t-SNE] Iteration 200: error = 63.1847343, gradient norm = 0.4952295 (50 iterations in 0.065s)
[t-SNE] Iteration 250: error = 63.8511581, gradient norm = 0.5513871 (50 iterations in 0.060s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 63.851158
[t-SNE] Iteration 300: error = 1.1934975, gradient norm = 0.0078173 (50 iterations in 0.054s)
[t-SNE] Iteration 350: error = 0.9162434, gradient norm = 0.0051715 (

In [200]:
def pre_process_for_plot(TSNE_results, df_with_image_names):
    """
    Combines the results of TSNE in a dataframe with the img names
    returns a dataframe with coords for every image
    """
    
    names = df_with_image_names[df_with_image_names.columns[0]]
    
    if TSNE_results.shape[1] == 2:
        df_t = pd.DataFrame(results, columns=['x', 'y'])
        df_t = df_t.assign(img = names)
    
    if TSNE_results.shape[1] == 3:
        df_t = pd.DataFrame(results, columns=['x', 'y', 'z'])
        df_t = df_t.assign(img = names)
        
    
    else:
        print("Dimensions don't allign")
        
    return df_t
        
viz_df = pre_process_for_plot(results, df)
viz_df.head(5)

Dimensions don't allign


,x,y,img
0,-0.929617,-6.900858,CH-1995-1
1,-2.553830,5.281172,CH-1995-3
2,-1.690055,-0.898614,E-1995-4
3,-1.723864,-2.341164,E-1995-7
4,-1.727573,-2.363153,E-1995-19


In [201]:
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.io import output_notebook
from bokeh.palettes import Viridis256

def plot_TNSE(pre_processed_df, width=1000, height=650, dot_size=10):
    """
    Function that plots the results from TNSE. Required df processed by def pre_process_for_plot as input
    Outputs a bokeh plot
    """
    
    output_notebook()

    source = ColumnDataSource(
            data=dict(
                x=pre_processed_df['x'],
                y=pre_processed_df['y'],
                desc=pre_processed_df['img'],
                color = Viridis256[:len(viz_df)]
            )
        )

    hover = HoverTool(tooltips=[("Image", "@desc")])

    p = figure(plot_width=width, plot_height=650, tools=[hover],
               title="TSNE results for the N_O_W_H_E_R_E dataset")

    p.circle('x', 'y', color='color', size=dot_size, source=source)

    show(p)
    
    
plot_TNSE(viz_df, width=1000)

Loading BokehJS ...

In [191]:
def write_csv(pre_processed_df, name):
    """
    Functions that exports the data from Bokeh plot to csv so it can be used on the website
    Returns a csv file
    """
    return pre_processed_df.to_csv(name + ".csv")


write_csv(viz_df, 'results_TSNE_2d')